In [134]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
my_url = 'https://www.hltv.org/matches?star=1'

page = requests.get(my_url, headers=headers)
page_html = page.text

soup = BeautifulSoup(page_html,"html.parser")

In [135]:
match_day_divs = soup.find_all('div', class_="match-day")

In [136]:
from datetime import datetime
from datetime import timedelta  
now = datetime.now()+ timedelta(days=1)

In [137]:
def GetMatchesForDay(match_day):
    matches = match_day.find_all('a')
    match_soups = {}
    for match in matches:
        #print(match["href"])
        my_url = "https://www.hltv.org" + match["href"]
        page = requests.get(my_url, headers=headers)
        page_html = page.text

        match_soup = soup = BeautifulSoup(page_html,"html.parser")
        match_soups[match["href"]] = (match_soup)
    return match_soups

In [138]:
#match_day = match_day_divs[1]
match_soups = {}
for match_day in match_day_divs:
    match_date_text = match_day.find_all('span')[0].text
    match_date = datetime.strptime(match_date_text, '%Y-%m-%d')
    if (now.date() == match_date.date()):
        match_soups = GetMatchesForDay(match_day)
        break

In [139]:
data = []
providers = ["geoprovider_bet365 betting_provider","ggbet-odds geoprovider_ggbet betting_provider", "pinnacle-odds geoprovider_pinnacle betting_provider", "geoprovider_unibet betting_provider", "geoprovider_betway betting_provider"]
for match_key in match_soups.keys():
    data_entry = []
    soup = match_soups[match_key]
    match_id = match_key.split('/')[2]
    data_entry.append(match_id)
    data_entry.append(match_key)
    for provider in providers:        
        betting_table = soup.find('div', class_=["betting standard-box"])
        row = betting_table.find('tr', class_=[provider])
        columns = row.find_all('td', class_ = ["odds-cell border-left"])
        if (len(columns) == 3):
            left = columns[0].find('a').text
            right = columns[2].find('a').text
            data_entry.append(left)
            data_entry.append(right)
            #print(provider, left.find('a').text, right.find('a').text)
        else:
            data_entry.append("1.0")
            data_entry.append("1.0")
    data.append(data_entry)

In [140]:
columns = ["matchId", "url", "bet365_1", "best365_2", "ggbet_1", "ggbet_2", "pinnacle_1", "pinnacle_2", "unibet_1", "unibet_2", "betway_1", "betway_2"]

In [141]:
file_name = "odds-" + datetime.strftime(now.date(), '%Y-%m-%d')

In [142]:
import csv
myFile = open(file_name + ".csv", 'w', newline='')
with myFile:
    writer = csv.writer(myFile)
    writer.writerow(columns)
    writer.writerows(data)